In [46]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Set Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = 'nachovelu'
os.environ['KAGGLE_KEY'] = 'eca89947e34630c8926c53427f4ad4da'

# Initialize the Kaggle API
api = KaggleApi()
api.authenticate()

# Download the dataset
dataset = 'parisrohan/credit-score-classification'
api.dataset_download_files(dataset, path='.', unzip=True)


Dataset URL: https://www.kaggle.com/datasets/parisrohan/credit-score-classification


In [47]:
import pandas as pd 

data_train = pd.read_csv(r'C:\Users\wipip\OneDrive\Documentos\GitHub\Proyecto-Clasificacion-de-Score-Crediticio-main\PROYECTO\src\train.csv')
data_test = pd.read_csv(r'C:\Users\wipip\OneDrive\Documentos\GitHub\Proyecto-Clasificacion-de-Score-Crediticio-main\PROYECTO\src\test.csv')

C:\Users\wipip\AppData\Local\Temp\ipykernel_25196\2209697490.py:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data_train = pd.read_csv(r'C:\Users\wipip\OneDrive\Documentos\GitHub\Proyecto-Clasificacion-de-Score-Crediticio-main\PROYECTO\src\train.csv')


In [48]:
print(data_test.info() , data_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        50000 non-null  object 
 1   Customer_ID               50000 non-null  object 
 2   Month                     50000 non-null  object 
 3   Name                      44985 non-null  object 
 4   Age                       50000 non-null  object 
 5   SSN                       50000 non-null  object 
 6   Occupation                50000 non-null  object 
 7   Annual_Income             50000 non-null  object 
 8   Monthly_Inhand_Salary     42502 non-null  float64
 9   Num_Bank_Accounts         50000 non-null  int64  
 10  Num_Credit_Card           50000 non-null  int64  
 11  Interest_Rate             50000 non-null  int64  
 12  Num_of_Loan               50000 non-null  object 
 13  Type_of_Loan              44296 non-null  object 
 14  Delay_

In [49]:
import sqlite3 

In [50]:
conn = sqlite3.connect('database_credit.db')

In [51]:
cursor = conn.cursor()

In [52]:
data_train.to_sql('training_set', conn, if_exists='append', index=False)
data_test.to_sql('testing_set', conn, if_exists='append', index=False)

50000

In [53]:
conn.commit()

In [54]:
query_1 = ''' 
SELECT name FROM sqlite_master WHERE type='table';

'''
cursor.execute(query_1)
tables = cursor.fetchall()

In [55]:
print("Tablas en la base de datos:")
for table in tables:
    print(table[0])

Tablas en la base de datos:
training_set
testing_set


In [56]:
tabla_prueba = pd.read_sql_query('SELECT * FROM training_set', conn)
tabla_prueba.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,None,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [57]:
query_2 = ''' 
SELECT ID, COUNT(*)
FROM training_set
GROUP BY ID
HAVING COUNT(*) > 1;
'''
duplicados_en_training_set = pd.read_sql_query(query_2, conn )

In [58]:
print(duplicados_en_training_set.shape)

(100000, 2)


Con este query podemos comprobar que no existen filas duplicadas dentro de nuestro data set de training

In [64]:
data_1 = pd.read_sql_query('SELECT * FROM training_set', conn)

In [65]:
for tabla in tables:
    print(f"\nEstructura de la tabla {tabla[0]}:")
    cursor.execute(f"PRAGMA table_info({tabla[0]});")
    columnas = cursor.fetchall()
    for columna in columnas:
        print(f"{columna[1]} - {columna[2]}")


Estructura de la tabla training_set:
ID - TEXT
Customer_ID - TEXT
Month - TEXT
Name - TEXT
Age - TEXT
SSN - TEXT
Occupation - TEXT
Annual_Income - TEXT
Monthly_Inhand_Salary - REAL
Num_Bank_Accounts - INTEGER
Num_Credit_Card - INTEGER
Interest_Rate - INTEGER
Num_of_Loan - TEXT
Type_of_Loan - TEXT
Delay_from_due_date - INTEGER
Num_of_Delayed_Payment - TEXT
Changed_Credit_Limit - TEXT
Num_Credit_Inquiries - REAL
Credit_Mix - TEXT
Outstanding_Debt - TEXT
Credit_Utilization_Ratio - REAL
Credit_History_Age - TEXT
Payment_of_Min_Amount - TEXT
Total_EMI_per_month - REAL
Amount_invested_monthly - TEXT
Payment_Behaviour - TEXT
Monthly_Balance - TEXT
Credit_Score - TEXT

Estructura de la tabla testing_set:
ID - TEXT
Customer_ID - TEXT
Month - TEXT
Name - TEXT
Age - TEXT
SSN - TEXT
Occupation - TEXT
Annual_Income - TEXT
Monthly_Inhand_Salary - REAL
Num_Bank_Accounts - INTEGER
Num_Credit_Card - INTEGER
Interest_Rate - INTEGER
Num_of_Loan - TEXT
Type_of_Loan - TEXT
Delay_from_due_date - INTEGER
Nu

In [66]:
conn.commit()

Por el momento cerramos el uso de la base de datos, una vez que analizamos la estructura de las dos tablas podemos tener una mejor idea de el tipo de datos con el que estamos trabajando, con ello podemos proceder a realizar un analisis descriptivo.

Vamos a empezar a describir el training set al cual llamamos como data_db una vez que lo llamamos desde nuestra base de datos.

In [67]:
data_1.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        200000 non-null  object 
 1   Customer_ID               200000 non-null  object 
 2   Month                     200000 non-null  object 
 3   Name                      180030 non-null  object 
 4   Age                       200000 non-null  object 
 5   SSN                       200000 non-null  object 
 6   Occupation                200000 non-null  object 
 7   Annual_Income             200000 non-null  object 
 8   Monthly_Inhand_Salary     169996 non-null  float64
 9   Num_Bank_Accounts         200000 non-null  int64  
 10  Num_Credit_Card           200000 non-null  int64  
 11  Interest_Rate             200000 non-null  int64  
 12  Num_of_Loan               200000 non-null  object 
 13  Type_of_Loan              177184 non-null  o

In [62]:
data_db.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,169996.000000,200000.000000,200000.000000,200000.000000,200000.000000,196070.000000,200000.000000,200000.000000
mean,4194.170850,17.091280,22.474430,72.466040,21.068780,27.754251,32.285173,1403.118217
std,3183.676803,117.404541,129.057087,466.421455,14.860067,193.176846,5.116862,8306.020505
min,303.645417,-1.000000,0.000000,1.000000,-5.000000,0.000000,20.000000,0.000000
25%,1625.558333,3.000000,4.000000,8.000000,10.000000,3.000000,28.052567,30.306660
50%,3093.745000,6.000000,5.000000,13.000000,18.000000,6.000000,32.305784,69.249473
75%,5957.448333,7.000000,7.000000,20.000000,28.000000,9.000000,36.496663,161.224249
max,15204.633333,1798.000000,1499.000000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


In [ ]:
def estadistica_descriptiva(df):
    for columna in df:
        